In [27]:
import json
import torch

In [28]:
# with open('../out/eval_results_text_inversion__nq.corr.sources.json', 'r') as f:
with open('../out/gtr-to-e5--trans/eval_results_text_inversion__nq.json', 'r') as f:
    eval_results = json.load(f)

In [29]:
eval_results.keys()

dict_keys(['pairs_of_source_w_aligned', 'data_to_invert'])

In [30]:
eval_results['pairs_of_source_w_aligned']['text_pairs'][0]

{'text': 'has been consistently named the best coffee in India by the Coffee Board of India\'s annual "Flavour of India – Fine Cup Award", ',
 'inv_text': 'is ranked among the Top 5 in India for winning The Annual Coffee Association of India Best Coffee Award for Overall flavour and quantity for The India Coffee Board.',
 'cosine': 0.9726678133010864}

In [31]:
texts, inv_texts = [], []
for pair in eval_results['pairs_of_source_w_aligned']['text_pairs']:
# for pair in eval_results['pairs_of_source']['text_pairs']:
# for pair in eval_results['pairs_of_target']['text_pairs']:
    # print(pair)
    text, inv_text = pair['text'], pair['inv_text']
    texts.append(text)
    inv_texts.append(inv_text)

In [32]:
def print_top_k(scores, _texts=texts, _inv_texts=inv_texts, top_k=10):
    # Find top-scored pairs
    top_k_indices = torch.argsort(scores, descending=True)[:top_k]
    
    # Print the corresponding pairs
    for i in top_k_indices:
        print(f'   >>{_texts[i]} \n   >>{_inv_texts[i]}')
        print("-------------------")

## Evaluate BERTScore

In [33]:
import evaluate

bertscore = evaluate.load("bertscore")
bertscore_results = bertscore.compute(predictions=inv_texts, references=texts, lang="en")
bertscore_results = torch.tensor(bertscore_results['f1'])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
print_top_k(-1 * bertscore_results)

   >>Raksha Sainyam Raksha Sainyam (രക്ഷാസൈന്യം) is the Malayalam term used for the Salvation Army in Kerala, India. Ratchannai Senai 
   >>Krishnasen Rita is the Kerala Sambhikary (Rakam) or Sri Lankan Sambhikary (Rakam) organization
-------------------
   >>Michalis Paraskevas Michalis Paraskevas (Greek: Μιχάλης Παρασκευάς), is a Cypriot lawyer and activist. His activism work primari 
   >>Petros Marriacicus is a Greek lawyer activist who has been since 1 January 2012, Marriacicus Marriacicus
-------------------
   >>he was in. Jonah Weiner of "Slate" magazine declared Swanson ""Parks and Recreation"<nowiki>'</nowiki>s secret weapon" and "vita 
   >>actor recalled: "Judy likened it to being among those for Parker-Senior and 'Wonski
-------------------
   >>Jesuit casuists who had been heavily attacked by Pascal in his "Provincial Letters") as "propositiones laxorum moralistarum" (pr 
   >>pseudo-Capanians of "" (""; d. 1120) had sent vilious cases to Cardinal François de la Blanc
---

In [35]:
bertscore_results.mean().item()

0.859453022480011

## Cosine Sim. of third-party embedding (Nomic)

In [36]:
# Cosine Sim. of third-party embedding
# nomic-ai/nomic-embed-text-v1.5

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
sentences = ['TSNE is a dimensionality reduction algorithm created by Laurens van Der Maaten']
embeddings = model.encode(sentences)
print(embeddings)


<All keys matched successfully>


[[ 5.11632562e-01  1.11381459e+00 -3.64767861e+00 -1.41410959e+00
   2.75067747e-01  3.66615057e-01  2.36130565e-01  1.41984844e+00
  -1.45862699e+00  5.46104729e-01 -2.60101855e-01  1.68805644e-01
   2.49645352e+00 -9.45616290e-02 -2.14303613e-01  1.68305025e-01
  -8.34194779e-01 -1.52528256e-01 -4.57077593e-01  2.39236876e-01
  -2.40685657e-01 -7.15821922e-01 -8.04216802e-01 -1.06679022e+00
   1.75600517e+00  2.77915210e-01 -2.44685747e-02  4.39256370e-01
   3.10012817e-01 -9.53131840e-02  1.88017714e+00 -1.52821147e+00
  -1.61872488e-02  9.36693996e-02 -9.22468364e-01  2.92257905e-01
  -9.56503022e-03  2.85559595e-01 -6.56725839e-03 -7.78208748e-02
  -5.26739478e-01 -6.45117819e-01 -1.15278649e+00 -4.98640448e-01
   5.72908401e-01  1.25191405e-01  2.79295202e-02 -1.85146064e-01
  -6.27781212e-01 -6.14796698e-01 -6.81977998e-03  2.63101608e-01
   5.06523132e-01 -1.70898885e-01  8.97038519e-01  4.74085420e-01
  -6.27662003e-01  7.93271840e-01  1.89658821e+00 -3.53785664e-01
   2.24065

In [37]:
# Calculate the similarity for each pair

# Embed and calc cosine
embeddings = model.encode(texts, convert_to_tensor=True)
inv_embeddings = model.encode(inv_texts, convert_to_tensor=True)

sims = torch.nn.functional.cosine_similarity(embeddings, inv_embeddings)

sims

tensor([0.9168, 0.7942, 0.9341,  ..., 0.7440, 0.7657, 0.8106], device='cuda:0')

In [38]:
print_top_k(sims)

   >>District of Columbia Court of Appeals The District of Columbia Court of Appeals is the highest court of the District of Columbia 
   >>Court of Appeals for the District of Columbia Court of Appeals for the District of Columbia is the Supreme court of the District of Columbia. The highest court
-------------------
   >>Haole Haole (; Hawaiian ) is a term used in Hawaii to refer to individuals who are not descendants of the ethnic native Hawaiian 
   >>"haole") is a term used in Hawaii for persons who are not native Hawaiian descendants, although the term "haole" may be
-------------------
   >>Chuck (season 4) The fourth season of "Chuck" was announced on May 13, 2010. Having initially ordered 13 episodes, NBC ordered a 
   >>2010 "NBC: Chuck" Season 4 was ordered for 13 episodes for its upcoming fourth season. Season 4 was announced on May 9, 2010 by The All
-------------------
   >>Scream (Avenged Sevenfold song) "Scream" is the fifth and final single from Avenged Sevenfold's sel

In [39]:
sims.mean().item()

0.8007254004478455